In [ ]:
import json
import requests #Hacer peticiones a APIS
from shapely.geometry  import shape, mapping


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
with open ('/content/boundingbox.geojson', 'r', encoding='utf-8') as archivo:
  miJSON = json.load(archivo)

In [ ]:
miGeometria = miJSON['features'][0]['geometry']
miPoligono = shape(miGeometria) #Convertir info del geojson en una geometria
miPoligono

In [ ]:
xmin, ymin, xmax, ymax = miPoligono.bounds

In [ ]:
#Contruir consulta para conectar a la API
url = "https://geoportal.jbb.gov.co/agc/rest/services/SIGAU/CensoArbol/MapServer/0/query"
parametros = {
    "f":"json",
    "geometry":f"{xmin}, {ymin}, {xmax}, {ymax}",
    "geometryType":"esriGeometryEnvelope",
    "spatialRel": "esriSpatialRelIntersects",
    "where":"1=1",
    "outFields":"*"
}

In [ ]:
respuesta = requests.get(url, parametros)

In [ ]:
if respuesta.status_code==200:          #Codigo 200 es que todo salió perfecto
  with open ('arboles.geojson', 'w', encoding='utf-8') as archivo:
    archivo.write(respuesta.text)
else:
  print ("Todo se derrumbó")

In [ ]:
import geopandas as gpd

geo1 = gpd.read_file("/content/drive/MyDrive/arboles.geojson")
geo2 = gpd.read_file("/content/drive/MyDrive/kennedy.geojson")

geo2=geo2.to_crs(geo1.crs)

arboles = geo1[geo1.intersects(geo2.union_all())]
arboles.to_file("arboles_kennedy.geojson", driver="GeoJSON")